<div style="background-image: url('https://www.dropbox.com/scl/fi/wdrnuojbnjx6lgfekrx85/mcnair.jpg?rlkey=wcbaw5au7vh5vt1g5d5x7fw8f&dl=1'); background-size: cover; background-position: center; height: 300px; display: flex; align-items: center; justify-content: center; color: white; text-shadow: 2px 2px 4px rgba(0,0,0,0.7); margin-bottom: 20px; position: relative;">
  <h1 style="text-align: center; font-size: 2.5em; margin: 0;">JGSB Python Workshop <br> Part 10: Simulation</h1>
  <div style="position: absolute; bottom: 10px; left: 15px; font-size: 0.9em; color: white; text-shadow: 2px 2px 4px rgba(0,0,0,0.7);">
    Authored by Kerry Back
  </div>
  <div style="position: absolute; bottom: 10px; right: 15px; text-align: right; font-size: 0.9em; color: white; text-shadow: 2px 2px 4px rgba(0,0,0,0.7);">
    Rice University, 9/6/2025
  </div>
</div>

## Exercise: Target-Date Fund Analysis

Create a dynamic asset allocation simulation that changes over time, similar to target-date retirement funds that become more conservative as retirement approaches.

**Your Task:**
1. Design a glide path that reduces stock allocation over time:
   - **Young (Years 1-10):** 90% stocks, 10% bonds
   - **Middle (Years 11-20):** Linear decrease from 90% to 60% stocks
   - **Pre-retirement (Years 21-30):** Linear decrease from 60% to 30% stocks
   - **Retirement (Years 31-40):** Stable at 30% stocks, 70% bonds

2. Use these asset parameters:
   - Stocks: 8% expected return, 18% volatility
   - Bonds: 4% expected return, 6% volatility  
   - Correlation: 0.3 between stocks and bonds

3. Compare the target-date strategy against:
   - Static aggressive portfolio (80% stocks throughout)
   - Static conservative portfolio (40% stocks throughout)

4. Analyze which strategy provides:
   - Best risk-adjusted returns (Sharpe ratio)
   - Lowest probability of large losses in final 10 years
   - Most consistent retirement outcomes

5. Visualize the changing asset allocation and compare final wealth distributions

In [ ]:
def simulate_portfolio_returns(asset_allocations, expected_returns, volatilities, 
                              correlation_matrix, initial_investment, 
                              num_years=10, num_simulations=1000, rebalance_frequency=12):
    """
    Simulate portfolio performance with multiple correlated assets
    
    Parameters:
    - asset_allocations: List of allocation percentages (must sum to 1.0)
    - expected_returns: List of annual expected returns for each asset
    - volatilities: List of annual volatilities for each asset  
    - correlation_matrix: Correlation matrix between assets
    - initial_investment: Starting portfolio value
    - num_years: Investment horizon
    - num_simulations: Number of scenarios
    - rebalance_frequency: Months between rebalancing (12 = annual)
    
    Returns:
    - Dictionary with simulation results and statistics
    """
    
    num_assets = len(asset_allocations)
    num_months = num_years * 12
    
    # Convert annual parameters to monthly
    monthly_returns = [r/12 for r in expected_returns]
    monthly_vols = [v/np.sqrt(12) for v in volatilities]
    
    # Generate correlated random returns using Cholesky decomposition
    L = np.linalg.cholesky(correlation_matrix)
    
    portfolio_values = np.zeros((num_simulations, num_months + 1))
    portfolio_values[:, 0] = initial_investment
    
    for sim in range(num_simulations):
        current_value = initial_investment
        asset_values = np.array(asset_allocations) * initial_investment
        
        for month in range(num_months):
            # Generate correlated random returns
            random_shocks = np.random.standard_normal(num_assets)
            correlated_shocks = L @ random_shocks
            
            # Calculate monthly returns for each asset
            monthly_asset_returns = [
                monthly_returns[i] + monthly_vols[i] * correlated_shocks[i]
                for i in range(num_assets)
            ]
            
            # Update asset values
            for i in range(num_assets):
                asset_values[i] *= (1 + monthly_asset_returns[i])
            
            current_value = np.sum(asset_values)
            
            # Rebalance if it's time
            if (month + 1) % rebalance_frequency == 0:
                asset_values = np.array(asset_allocations) * current_value
            
            portfolio_values[sim, month + 1] = current_value
    
    # Calculate key statistics
    final_values = portfolio_values[:, -1]
    annual_returns = (final_values / initial_investment) ** (1/num_years) - 1
    
    results = {
        'portfolio_paths': portfolio_values,
        'final_values': final_values,
        'annual_returns': annual_returns,
        'statistics': {
            'mean_final_value': np.mean(final_values),
            'median_final_value': np.median(final_values),
            'mean_annual_return': np.mean(annual_returns),
            'volatility': np.std(annual_returns),
            'sharpe_ratio': np.mean(annual_returns) / np.std(annual_returns),
            'var_5pct': np.percentile(final_values, 5),
            'var_10pct': np.percentile(final_values, 10),
            'prob_loss': np.mean(final_values < initial_investment),
            'prob_double': np.mean(final_values >= 2 * initial_investment)
        }
    }
    
    return results

# Example: Three-asset portfolio analysis
np.random.seed(42)

# Define three portfolios to compare
portfolios = {
    'Conservative': {
        'allocations': [0.3, 0.6, 0.1],  # 30% stocks, 60% bonds, 10% cash
        'description': '30% Stocks, 60% Bonds, 10% Cash'
    },
    'Balanced': {
        'allocations': [0.6, 0.35, 0.05],  # 60% stocks, 35% bonds, 5% cash
        'description': '60% Stocks, 35% Bonds, 5% Cash'
    },
    'Aggressive': {
        'allocations': [0.85, 0.15, 0.0],  # 85% stocks, 15% bonds, 0% cash
        'description': '85% Stocks, 15% Bonds'
    }
}

# Asset characteristics (annual)
asset_names = ['Stocks', 'Bonds', 'Cash']
expected_returns = [0.10, 0.04, 0.02]  # 10%, 4%, 2% annual returns
volatilities = [0.20, 0.08, 0.01]      # 20%, 8%, 1% annual volatility

# Correlation matrix
correlation_matrix = np.array([
    [1.00, 0.20, 0.05],  # Stocks vs Bonds: 0.2, Stocks vs Cash: 0.05
    [0.20, 1.00, 0.10],  # Bonds vs Cash: 0.1
    [0.05, 0.10, 1.00]
])

# Simulation parameters
initial_investment = 100000  # $100,000 initial investment
num_years = 20              # 20-year horizon
num_simulations = 5000      # 5,000 scenarios
rebalance_frequency = 12    # Annual rebalancing

# Run simulations for all portfolios
portfolio_results = {}
for name, portfolio in portfolios.items():
    print(f"Running simulation for {name} portfolio...")
    results = simulate_portfolio_returns(
        portfolio['allocations'], expected_returns, volatilities,
        correlation_matrix, initial_investment, num_years, 
        num_simulations, rebalance_frequency
    )
    portfolio_results[name] = results

# Display results comparison
print("PORTFOLIO PERFORMANCE COMPARISON")
print("="*80)
print(f"{'Portfolio':<12} {'Mean Final':<12} {'Median Final':<13} {'Annual Ret':<11} {'Volatility':<11} {'Sharpe':<7} {'5% VaR':<10}")
print("-" * 80)

for name, results in portfolio_results.items():
    stats = results['statistics']
    print(f"{name:<12} ${stats['mean_final_value']:<11,.0f} ${stats['median_final_value']:<12,.0f} {stats['mean_annual_return']:<10.1%} {stats['volatility']:<10.1%} {stats['sharpe_ratio']:<6.2f} ${stats['var_5pct']:<9,.0f}")

print(f"\nRISK ANALYSIS:")
print(f"{'Portfolio':<12} {'Prob of Loss':<13} {'Prob of 2x':<12} {'10% VaR':<12}")
print("-" * 50)
for name, results in portfolio_results.items():
    stats = results['statistics']
    print(f"{name:<12} {stats['prob_loss']:<12.1%} {stats['prob_double']:<11.1%} ${stats['var_10pct']:<11,.0f}")

# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Sample portfolio paths
ax1 = axes[0, 0]
years = np.arange(0, num_years + 1/12, 1/12)[:num_years*12 + 1]
colors = ['blue', 'green', 'red']

for i, (name, results) in enumerate(portfolio_results.items()):
    # Show 5 sample paths for each portfolio
    for j in range(5):
        ax1.plot(years, results['portfolio_paths'][j], 
                color=colors[i], alpha=0.3, linewidth=0.8)
    
    # Add mean path
    mean_path = np.mean(results['portfolio_paths'], axis=0)
    ax1.plot(years, mean_path, color=colors[i], linewidth=2, label=f'{name} Mean')

ax1.set_title('Sample Portfolio Growth Paths')
ax1.set_xlabel('Years')
ax1.set_ylabel('Portfolio Value ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Final value distributions
ax2 = axes[0, 1]
for i, (name, results) in enumerate(portfolio_results.items()):
    ax2.hist(results['final_values'], bins=50, alpha=0.6, 
            color=colors[i], label=name, density=True)

ax2.set_title('Distribution of Final Portfolio Values')
ax2.set_xlabel('Final Value ($)')
ax2.set_ylabel('Density')
ax2.legend()

# Plot 3: Risk-return scatter
ax3 = axes[1, 0]
returns = [results['statistics']['mean_annual_return'] for results in portfolio_results.values()]
volatilities_calc = [results['statistics']['volatility'] for results in portfolio_results.values()]
names = list(portfolio_results.keys())

ax3.scatter(volatilities_calc, returns, s=100, c=colors)
for i, name in enumerate(names):
    ax3.annotate(name, (volatilities_calc[i], returns[i]), 
                xytext=(5, 5), textcoords='offset points')

ax3.set_title('Risk-Return Profile')
ax3.set_xlabel('Volatility (Annual Return Std Dev)')
ax3.set_ylabel('Mean Annual Return')
ax3.grid(True, alpha=0.3)

# Plot 4: Value at Risk comparison
ax4 = axes[1, 1]
var_5pct = [results['statistics']['var_5pct'] for results in portfolio_results.values()]
var_10pct = [results['statistics']['var_10pct'] for results in portfolio_results.values()]

x = np.arange(len(names))
width = 0.35

ax4.bar(x - width/2, var_5pct, width, label='5% VaR', alpha=0.7, color='red')
ax4.bar(x + width/2, var_10pct, width, label='10% VaR', alpha=0.7, color='orange')
ax4.axhline(initial_investment, color='black', linestyle='--', 
           label='Initial Investment')

ax4.set_title('Value at Risk Comparison')
ax4.set_xlabel('Portfolio')
ax4.set_ylabel('Portfolio Value ($)')
ax4.set_xticks(x)
ax4.set_xticklabels(names)
ax4.legend()

plt.tight_layout()
plt.show()

# Summary insights
print(f"\nKEY INSIGHTS:")
print(f"• Higher stock allocation increases both expected returns and volatility")
print(f"• Conservative portfolio has lowest downside risk but also lowest upside potential")
print(f"• Aggressive portfolio offers best long-term growth but highest short-term risk")
print(f"• All portfolios benefit from diversification and rebalancing")
print(f"• Risk tolerance should match investment timeline and financial goals")

# Investment Portfolio Risk Analysis

Portfolio management requires understanding how different asset combinations behave under various market conditions. Monte Carlo simulation helps investors and financial advisors assess portfolio risk, optimize asset allocation, and set realistic return expectations.

**Key Portfolio Concepts:**
- **Asset Allocation:** Percentage invested in stocks, bonds, and other assets
- **Correlation:** How asset returns move together during market events
- **Rebalancing:** Periodically adjusting allocations back to target weights
- **Risk-Return Tradeoff:** Higher expected returns typically come with higher volatility
- **Value at Risk (VaR):** Maximum expected loss over a given time period with specified confidence

**Simulation Applications:**
- Testing portfolio performance across thousands of market scenarios
- Calculating probability of meeting financial goals
- Optimizing asset allocation for desired risk levels
- Stress testing portfolios against market crashes

## Exercise: Restaurant Chain Expansion

You're analyzing sales forecasts for a new restaurant chain location. Create a simulation that incorporates both growth and declining phases as the restaurant matures.

**Your Task:**
1. Model restaurant lifecycle with three phases:
   - **Ramp-up (Months 1-6):** Starting at 50% of target sales, growing to 100%
   - **Growth (Months 7-24):** 2% monthly growth from established base
   - **Maturity (Months 25-36):** Stable sales with only random variation
2. Include these realistic factors:
   - Base monthly sales target: $80,000
   - Seasonal patterns for restaurants (lower in Jan/Feb, higher in summer)
   - Random monthly variation: 20% standard deviation
   - Competition impact: 5% sales decline starting in month 18
3. Calculate key business metrics:
   - Probability of profitable first year (break-even: $720,000)
   - Expected payback period for $500,000 initial investment
   - Range of 3-year cumulative sales (10th to 90th percentile)
4. Create visualizations showing the lifecycle pattern and uncertainty bands

In [ ]:
def simulate_monthly_sales(base_monthly_sales, annual_growth_rate, seasonal_factors, 
                          economic_volatility, num_years=3, num_simulations=1000):
    """
    Simulate monthly sales with growth, seasonality, and random variation
    
    Parameters:
    - base_monthly_sales: Average monthly sales in first year
    - annual_growth_rate: Expected yearly growth (e.g., 0.05 for 5%)
    - seasonal_factors: List of 12 seasonal multipliers (Jan=0, Dec=11)
    - economic_volatility: Standard deviation of random economic shocks
    - num_years: Forecast horizon in years
    - num_simulations: Number of scenarios to simulate
    
    Returns:
    - Array of shape (num_simulations, num_years*12) with monthly sales
    """
    
    num_months = num_years * 12
    all_simulations = np.zeros((num_simulations, num_months))
    
    for sim in range(num_simulations):
        monthly_sales = np.zeros(num_months)
        
        for month in range(num_months):
            # Calculate year and month for growth and seasonality
            year = month // 12
            month_of_year = month % 12
            
            # Base sales with annual growth
            base_sales = base_monthly_sales * (1 + annual_growth_rate) ** year
            
            # Apply seasonal factor
            seasonal_sales = base_sales * seasonal_factors[month_of_year]
            
            # Add random economic shock
            economic_shock = np.random.normal(1, economic_volatility)
            final_sales = seasonal_sales * economic_shock
            
            # Ensure sales don't go negative
            monthly_sales[month] = max(final_sales, 0)
        
        all_simulations[sim] = monthly_sales
    
    return all_simulations

# Example: E-commerce retailer forecasting
np.random.seed(42)

# Parameters for an online retail business
base_monthly_sales = 150000    # $150k average monthly sales
annual_growth_rate = 0.08      # 8% annual growth expected
economic_volatility = 0.15     # 15% monthly volatility

# Seasonal factors (retail pattern with holiday peaks)
seasonal_factors = [
    0.85,  # January (post-holiday slump)
    0.90,  # February  
    0.95,  # March
    1.00,  # April
    1.05,  # May
    1.00,  # June
    0.95,  # July
    1.00,  # August
    1.05,  # September
    1.10,  # October
    1.25,  # November (Black Friday)
    1.40   # December (Holiday season)
]

num_years = 3
num_simulations = 5000

# Run simulation
sales_simulations = simulate_monthly_sales(
    base_monthly_sales, annual_growth_rate, seasonal_factors,
    economic_volatility, num_years, num_simulations
)

# Analyze results
total_months = num_years * 12
monthly_stats = pd.DataFrame({
    'Month': range(1, total_months + 1),
    'Mean_Sales': np.mean(sales_simulations, axis=0),
    'Median_Sales': np.median(sales_simulations, axis=0),
    'P10_Sales': np.percentile(sales_simulations, 10, axis=0),
    'P90_Sales': np.percentile(sales_simulations, 90, axis=0),
    'Std_Sales': np.std(sales_simulations, axis=0)
})

print("SALES FORECASTING SIMULATION RESULTS")
print("="*50)
print(f"Base monthly sales: ${base_monthly_sales:,}")
print(f"Annual growth rate: {annual_growth_rate:.1%}")
print(f"Economic volatility: {economic_volatility:.1%}")
print(f"Forecast horizon: {num_years} years")
print(f"Number of simulations: {num_simulations:,}")

# Annual summaries
for year in range(num_years):
    year_start = year * 12
    year_end = (year + 1) * 12
    annual_sales = np.sum(sales_simulations[:, year_start:year_end], axis=1)
    
    print(f"\nYEAR {year + 1} FORECAST:")
    print(f"Mean annual sales: ${np.mean(annual_sales):,.0f}")
    print(f"10th percentile: ${np.percentile(annual_sales, 10):,.0f}")
    print(f"90th percentile: ${np.percentile(annual_sales, 90):,.0f}")
    print(f"Probability of exceeding ${base_monthly_sales * 12 * (1 + annual_growth_rate)**year * 1.1:,.0f}: {np.mean(annual_sales > base_monthly_sales * 12 * (1 + annual_growth_rate)**year * 1.1):.1%}")

# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Sample sales paths
ax1 = axes[0, 0]
months = range(1, total_months + 1)
for i in range(10):  # Show 10 sample paths
    ax1.plot(months, sales_simulations[i], alpha=0.3, color='blue')
ax1.plot(months, monthly_stats['Mean_Sales'], color='red', linewidth=2, label='Mean')
ax1.fill_between(months, monthly_stats['P10_Sales'], monthly_stats['P90_Sales'], 
                alpha=0.2, color='red', label='10th-90th percentile')
ax1.set_title('Sample Sales Paths (10 simulations)')
ax1.set_xlabel('Month')
ax1.set_ylabel('Monthly Sales ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Distribution of Year 1 total sales
ax2 = axes[0, 1]
year1_totals = np.sum(sales_simulations[:, :12], axis=1)
ax2.hist(year1_totals, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
ax2.axvline(np.mean(year1_totals), color='red', linestyle='--', 
           label=f'Mean: ${np.mean(year1_totals):,.0f}')
ax2.set_title('Distribution of Year 1 Total Sales')
ax2.set_xlabel('Annual Sales ($)')
ax2.set_ylabel('Frequency')
ax2.legend()

# Plot 3: Monthly forecast with confidence bands
ax3 = axes[1, 0]
ax3.plot(months, monthly_stats['Mean_Sales'], color='blue', linewidth=2, label='Mean Forecast')
ax3.fill_between(months, monthly_stats['P10_Sales'], monthly_stats['P90_Sales'], 
                alpha=0.3, color='blue', label='80% Confidence Interval')
ax3.set_title('Monthly Sales Forecast with Uncertainty')
ax3.set_xlabel('Month')
ax3.set_ylabel('Monthly Sales ($)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Seasonal pattern validation
ax4 = axes[1, 1]
monthly_averages = [np.mean(sales_simulations[:, month::12]) for month in range(12)]
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
ax4.bar(month_names, monthly_averages, color='lightgreen', alpha=0.7)
ax4.set_title('Average Monthly Sales by Season')
ax4.set_xlabel('Month')
ax4.set_ylabel('Average Sales ($)')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Summary statistics table
print(f"\nMONTHLY FORECAST SUMMARY (First 12 Months):")
print("Month | Mean Sales | 10th Pct | 90th Pct | Std Dev")
print("-" * 55)
for i in range(12):
    print(f"{month_names[i]:5s} | ${monthly_stats.iloc[i]['Mean_Sales']:8,.0f} | ${monthly_stats.iloc[i]['P10_Sales']:7,.0f} | ${monthly_stats.iloc[i]['P90_Sales']:7,.0f} | ${monthly_stats.iloc[i]['Std_Sales']:6,.0f}")

# Monte Carlo Sales Forecasting

Sales forecasting is challenging because demand is influenced by many random factors: economic conditions, competitor actions, seasonal effects, and consumer behavior. Monte Carlo simulation captures this uncertainty and provides realistic ranges for business planning.

**Key Components:**
- **Base Demand:** Expected sales level without random variation
- **Seasonal Patterns:** Predictable fluctuations throughout the year
- **Economic Factors:** Random impacts from market conditions
- **Growth Trends:** Long-term increases or decreases in demand
- **Random Shocks:** Unexpected events that affect sales

**Business Value:** Helps companies set realistic sales targets, plan inventory, allocate marketing budgets, and assess the probability of meeting financial goals.

## Exercise: Personal Retirement Planning

Create your own retirement simulation with different scenarios to understand the impact of key decisions.

**Your Task:**
1. Modify the simulation function to include salary growth (contributions increase by 3% annually)
2. Compare three scenarios:
   - **Conservative:** 5% return, 10% volatility, $300/month starting contribution
   - **Moderate:** 7% return, 15% volatility, $500/month starting contribution  
   - **Aggressive:** 9% return, 20% volatility, $700/month starting contribution
3. For each scenario, calculate:
   - Probability of reaching $500k, $1M, and $2M
   - 10th percentile (worst case) and 90th percentile (best case) outcomes
4. Create a comparison chart showing the three distributions
5. Which scenario would you choose and why?

**Bonus:** Add the impact of a one-time inheritance of $50,000 received after 20 years.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def simulate_retirement_account(initial_balance, monthly_contribution, annual_return_mean, 
                              annual_return_std, years, num_simulations=1000):
    """
    Simulate retirement account growth with random returns
    
    Parameters:
    - initial_balance: Starting account value
    - monthly_contribution: Monthly deposit amount
    - annual_return_mean: Expected annual return (e.g., 0.07 for 7%)
    - annual_return_std: Standard deviation of returns (e.g., 0.15 for 15%)
    - years: Investment time horizon
    - num_simulations: Number of scenarios to simulate
    
    Returns:
    - Array of final account balances for each simulation
    """
    
    final_balances = np.zeros(num_simulations)
    
    for sim in range(num_simulations):
        balance = initial_balance
        
        for year in range(years):
            # Generate random annual return
            annual_return = np.random.normal(annual_return_mean, annual_return_std)
            
            # Add monthly contributions throughout the year
            for month in range(12):
                balance += monthly_contribution
                # Apply monthly return (annual return / 12)
                monthly_return = annual_return / 12
                balance *= (1 + monthly_return)
        
        final_balances[sim] = balance
    
    return final_balances

# Example: 25-year-old planning for retirement at 65
np.random.seed(42)

# Parameters
initial_balance = 10000        # Starting with $10,000
monthly_contribution = 500     # $500 per month
annual_return_mean = 0.07      # 7% expected annual return
annual_return_std = 0.15       # 15% volatility
years = 40                     # 40 years until retirement
num_simulations = 10000        # Run 10,000 scenarios

# Run simulation
results = simulate_retirement_account(
    initial_balance, monthly_contribution, annual_return_mean, 
    annual_return_std, years, num_simulations
)

# Analyze results
print("RETIREMENT ACCOUNT SIMULATION RESULTS")
print("="*50)
print(f"Initial balance: ${initial_balance:,}")
print(f"Monthly contribution: ${monthly_contribution:,}")
print(f"Investment period: {years} years")
print(f"Expected annual return: {annual_return_mean:.1%}")
print(f"Return volatility: {annual_return_std:.1%}")
print(f"Number of simulations: {num_simulations:,}")

print(f"\nFINAL BALANCE STATISTICS:")
print(f"Mean: ${np.mean(results):,.0f}")
print(f"Median: ${np.median(results):,.0f}")
print(f"Standard deviation: ${np.std(results):,.0f}")
print(f"Minimum: ${np.min(results):,.0f}")
print(f"Maximum: ${np.max(results):,.0f}")

# Calculate percentiles
percentiles = [10, 25, 50, 75, 90, 95, 99]
print(f"\nPERCENTILE ANALYSIS:")
for p in percentiles:
    value = np.percentile(results, p)
    print(f"{p:2d}th percentile: ${value:,.0f}")

# Calculate probability of reaching $1 million
million_prob = np.mean(results >= 1000000) * 100
print(f"\nProbability of reaching $1 million: {million_prob:.1f}%")

# Visualize results
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.hist(results, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
plt.title('Distribution of Final Account Balances')
plt.xlabel('Final Balance ($)')
plt.ylabel('Frequency')
plt.axvline(np.mean(results), color='red', linestyle='--', label=f'Mean: ${np.mean(results):,.0f}')
plt.axvline(np.median(results), color='green', linestyle='--', label=f'Median: ${np.median(results):,.0f}')
plt.legend()

plt.subplot(2, 2, 2)
plt.boxplot(results)
plt.title('Box Plot of Final Balances')
plt.ylabel('Final Balance ($)')

plt.subplot(2, 2, 3)
# Show sample paths
sample_paths = []
for i in range(5):
    np.random.seed(i)
    balance = initial_balance
    path = [balance]
    
    for year in range(years):
        annual_return = np.random.normal(annual_return_mean, annual_return_std)
        for month in range(12):
            balance += monthly_contribution
            monthly_return = annual_return / 12
            balance *= (1 + monthly_return)
        path.append(balance)
    
    sample_paths.append(path)
    plt.plot(range(years + 1), path, alpha=0.7)

plt.title('Sample Account Growth Paths')
plt.xlabel('Years')
plt.ylabel('Account Balance ($)')

plt.subplot(2, 2, 4)
# Cumulative probability plot
sorted_results = np.sort(results)
cumulative_prob = np.arange(1, len(sorted_results) + 1) / len(sorted_results)
plt.plot(sorted_results, cumulative_prob)
plt.title('Cumulative Probability Distribution')
plt.xlabel('Final Balance ($)')
plt.ylabel('Cumulative Probability')
plt.axhline(0.5, color='red', linestyle='--', alpha=0.7)
plt.axvline(np.median(results), color='red', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# Retirement Account Growth Simulation

Retirement planning involves many uncertain factors: investment returns, inflation rates, contribution changes, and market volatility. Monte Carlo simulation helps us understand the range of possible retirement outcomes and the probability of meeting financial goals.

**Key Variables:**
- **Initial Balance:** Starting account value
- **Monthly Contributions:** Regular deposits (may increase with salary)
- **Annual Return:** Investment performance (varies randomly each year)
- **Time Horizon:** Years until retirement
- **Market Volatility:** Standard deviation of returns

**Business Insight:** This simulation helps financial advisors show clients the impact of different contribution levels and risk tolerances on retirement outcomes.

# Introduction to Monte Carlo Simulation

Monte Carlo simulation is a powerful computational technique that uses random sampling to model complex systems and analyze uncertainty. Named after the famous Monte Carlo casino, this method generates thousands of random scenarios to understand the range of possible outcomes and their probabilities.

**Key Concepts:**
- **Random Sampling:** Using probability distributions to generate realistic scenarios
- **Law of Large Numbers:** More simulations lead to more accurate estimates
- **Uncertainty Quantification:** Understanding the range and likelihood of outcomes
- **Risk Assessment:** Evaluating best-case, worst-case, and most likely scenarios

**Business Applications:**
- Financial planning and investment analysis
- Project risk management and budgeting
- Sales forecasting with market uncertainty
- Supply chain optimization
- Insurance and actuarial modeling

**Why Simulation Matters in Business:**
- Real-world systems are too complex for analytical solutions
- Captures the impact of multiple random factors simultaneously
- Provides confidence intervals and risk measures
- Supports data-driven decision making under uncertainty